In [0]:
spark.conf.set("fs.s3a.access.key", "")
spark.conf.set("fs.s3a.secret.key", "")
spark.conf.set("fs.s3a.endpoint", "s3.amazonaws.com")

In [0]:
import time

from pyspark.sql import SparkSession
from pyspark.sql.functions import row_number, current_timestamp, col, hour
from pyspark.sql.window import Window

def generate_output_data(ref_data, validated_data, symbol_data, files_per_day, date):
    spark = validated_data.sparkSession  

    (
        currency_data, exchange_data, order_types_data, sides_data,
        transaction_types_data, order_statuses_data, mics_data
    ) = ref_data
    
    # Join validated_data with reference tables
    output_data = (
        validated_data
        .join(transaction_types_data.selectExpr("transaction_type", "transaction_type_id"), "transaction_type", "left")
        .join(mics_data.selectExpr("mic_code", "mic_id"), "mic_code", "left")
        .join(order_statuses_data.selectExpr("order_status", "order_status_id"), "order_status", "left")
        .join(sides_data.selectExpr("side", "side_id"), "side", "left")
        .join(order_types_data.selectExpr("order_type_name as order_type", "order_type_id"), "order_type", "left")
        .join(currency_data.selectExpr("currency_name", "currency_id"), "currency_name", "left")
        .join(exchange_data.selectExpr("exchange_code", "exchange_id"), "exchange_code", "left")
    )

    # Add transaction_hour column from transaction_timestamp
    output_data = output_data.withColumn("transaction_hour", hour(col("transaction_timestamp")))

    # Define column order
    column_order = [
        "transaction_id", "transaction_parent_id",
        "transaction_timestamp", "transaction_hour", "transaction_type_id", "mic_id",
        "order_status_id", "side_id", "order_type_id", "symbol",
        "isin", "price", "quantity","adv30", "trader_id", "broker_id",
        "exchange_id", "currency_id",
        "creation_time", "last_update_time", "validation_flag"
    ]

    # Add timestamps
    output_data = (
        output_data
        .withColumn("creation_time", current_timestamp())
        .withColumn("last_update_time", current_timestamp())
        .select(*column_order)
    )

    # output_data = output_data.repartitionByRange(files_per_day, "transaction_internal_id")
    output_data = output_data.repartition(files_per_day)
    output_path = "output_data"
    print(output_data.count())
    return output_data

start_time = time.time()
# Load DataFrames from Temporary Views
validated_data = spark.sql("SELECT * FROM global_temp.validated_data_view")
symbol_data = spark.sql("SELECT * FROM global_temp.symbol_data_view")
currency_data = spark.sql("SELECT * FROM global_temp.currency_data_view")
exchange_data = spark.sql("SELECT * FROM global_temp.exchange_data_view")
order_types_data = spark.sql("SELECT * FROM global_temp.order_types_data_view")
sides_data = spark.sql("SELECT * FROM global_temp.sides_data_view")
transaction_types_data = spark.sql("SELECT * FROM global_temp.transaction_types_data_view")
order_statuses_data = spark.sql("SELECT * FROM global_temp.order_statuses_data_view")
mics_data = spark.sql("SELECT * FROM global_temp.mics_data_view")
ref_data = (
    currency_data,
    exchange_data,
    order_types_data,
    sides_data,
    transaction_types_data,
    order_statuses_data,
    mics_data,
)
# os.makedirs("output_data", exist_ok=True)
output_data = generate_output_data(ref_data, validated_data,symbol_data, 20, "2025-03-17")
end_time = time.time()

# Calculate execution time
execution_time = end_time - start_time
print(f"Execution Time output data generation: {execution_time:.2f} seconds")

100000
Execution Time output data generation: 13.01 seconds


In [0]:
from pyspark.sql.functions import col, date_format
from delta.tables import DeltaTable

# Extract date and hour dynamically from transaction_timestamp
output_data = output_data.withColumn("date", date_format(col("transaction_timestamp"), "yyyy-MM-dd")) \
                         .withColumn("hour", date_format(col("transaction_timestamp"), "HH"))

# Write data partitioned by date and hour to S3 Delta Lake
output_data.write.format("delta") \
    .mode("append") \
    .partitionBy("date", "hour") \
    .option("path", "s3://delta-lake-etl/output_data_delta/") \
    .save()

# Optimize with Z-Ordering
# delta_table = DeltaTable.forPath(spark, "s3://delta-lake-etl/output_data_delta/")
# delta_table.optimize().executeZOrderBy("transaction_timestamp")
